### 0. Hello!

My first visit to Kaggle was about a year ago, and set a goal to upload a posting like other competitive smart people on the board. This is my first posting. Finally achieved sth in 2017! Any comments will be appreciated! Thanks :)


## 1. Introduction
- Preprocessed data: Drop columns, Scaling numbers, Over-sampling
- Logistic Regression, Decision Tree, Random Forest
- Threshold change

In [ ]:
import numpy as np
import pandas as pd
import pandas as pd
import matplotlib as mpl
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
%matplotlib inline

In [ ]:
def loadData(File):
    data = pd.read_csv(File)
    print("Top two records are: \n", data.head(2))
    #print("Description: \n", data.describe())
    return data
data = loadData("../input/creditcard.csv")

In [ ]:
def showColDist(colname, data):
    data = pd.DataFrame(data)
    return sns.countplot(x = colname, data = data )
showColDist("Class", data) # Showing high imbalance

In [ ]:
def dropCol(colname, axis):
    return data.drop(colname, axis = axis)
data = dropCol('Time', 1) # Firstly dropped Time, becasue

In [ ]:
def showDistplot(data):
    return sns.distplot(data)
showDistplot(data['Amount']) 

In [ ]:
def checkEachCol(data):
    List_col = list(data.columns.values)
    for i in List_col:
        print("====>",i,"<====")
        print(data[i].describe())
checkEachCol(data)

In [ ]:
def letsNorm(col):
    #from sklearn.preprocessing import StandardScaler
    inp = col.values.reshape(-1,1)
    scaler = StandardScaler()
    scaledData = scaler.fit_transform(inp)
    print("Newly normalized data : \n", scaler.fit_transform(inp),"This will be added, too.")
    data['Normalied'] = scaledData
    return data
data = letsNorm(data["Amount"])

In [ ]:
data.head(2)

In [ ]:
data = dropCol('Amount', 1) 
data.head(2)

## 2-1. Loading and spliting data

In [ ]:
def Xy_split(data):
    y = data['Class']
    List_col = list(data.columns.values)
    print(List_col)
    X_col = [x for x in List_col if x != "Class"]
    print(X_col)
    #print(X_col)
    X = data.loc[:,X_col]
    return X,y
X, y = Xy_split(data)

In [ ]:
X.head(3)

In [ ]:
y.head(3)

> ## 2-2. Over-sampling

In [ ]:
from imblearn.over_sampling import RandomOverSampler
from collections import Counter

In [ ]:
def randomOverSampling(X, y):
    #from imblearn.over_sampling import RandomOverSampler
    #from collections import Counter
    ros = RandomOverSampler()
    X_resampled, y_resampled = ros.fit_sample(X,y)
    #print(sorted(Counter(y_resampled).items()))
    return X_resampled, y_resampled

In [ ]:
X_big = X.copy()
y_big = y.copy()

In [ ]:
X_res , y_res = randomOverSampling(X_big, y_big)
print(" The # of original data is ", len(X_big),".\n","The # of the over-sampled one is ", len(X_res),".")

In [ ]:
sns.countplot(y_res)

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X_res, y_res, test_size = 0.25, random_state = 23)

In [ ]:
y_result = pd.DataFrame(y_test)
y_result.rename(columns={0: 'y_test'}, inplace=True)
y_result.head()

### 3-1. Linear Regression
---

####  `sklearn.linear_model.LinearRegression`
---****

In [ ]:
from sklearn.linear_model import LogisticRegression
LogisticReg = LogisticRegression()
LogisticReg.fit(X= X_train, y = y_train)
LogisticReg.predict(X = X_test) 
y_pred_lr = LogisticReg.predict(X = X_test)
print(y_pred_lr)

In [ ]:
y_result["Logistic_reg"] = y_pred_lr

In [ ]:
LogisticReg.score(X=X_test,y=y_test)

In [ ]:
y_result.head()

#### 3-2. Evaluation : Confusion Matrix 
---

In [ ]:
from sklearn.metrics import confusion_matrix
confusion_matrix(y_pred = LogisticReg.predict(X_test), y_true = y_test)
# [[ true negative , false positive],
# [false negative], [true positive]]
#print(LogisticReg.score(X=X_test,y=y_test))

### 3-1. Decision Tree
---

#### `sklearn.tree.DecisionTreeClassifier`
---**

In [ ]:
from sklearn.tree import DecisionTreeClassifier
class_weight = 'balanced'
max_depth = 5
# dt_model 
dt_model = DecisionTreeClassifier(class_weight = class_weight, max_depth = max_depth, criterion = 'entropy')

In [ ]:
dt_model.fit(X_train,y_train)

In [ ]:
dt_model.predict(X_test)

In [ ]:
dt_model.score(X = X_test, y = y_test)

In [ ]:
y_result['DecisionTree'] = dt_model.predict(X_test)

In [ ]:
y_result.head(10)

#### 3-2. Evaluation : Confusion Matrix 
* ---

In [ ]:
confusion_matrix(y_pred = dt_model.predict(X_test), y_true = y_test)

### 4-1. Random Forest
---

#### `sklearn.ensemble.RandomForestClassifier`
---

In [ ]:
from sklearn.ensemble import RandomForestClassifier
n_estimators = [3,15,75]
class_weight = 'balanced'
max_depth = 5

In [ ]:
# rf_model
for i in n_estimators:
    rf_model = RandomForestClassifier(n_estimators = i, class_weight = class_weight, max_depth = max_depth)
    rf_model.fit(X_train, y_train)
    rf_model.predict(X_test)
    print("rf_model score with # est. {}:".format(i), rf_model.score(X_test, y_test))
    y_result['rf-{}'.format(i)] = rf_model.predict(X_test)

In [ ]:
rf_model = RandomForestClassifier(n_estimators = 15, class_weight = class_weight, max_depth = max_depth)

In [ ]:
rf_model.fit(X_train, y_train)

In [ ]:
rf_model.predict(X_test)

In [ ]:
rf_model.score(X = X_test, y = y_test)

#### 4-2. Evaluation : Confusion Matrix 
* ---

In [ ]:
confusion_matrix(y_pred = rf_model.predict(X_test), y_true = y_test)

#### 5-1. Reports: Over-sampled, balanced test data
---

In [ ]:
from sklearn.metrics import classification_report

print("Logistic Regression\n",classification_report(y_test, y_pred =LogisticReg.predict(X = X_test)))
print("Decision Tree\n",classification_report(y_test, y_pred =dt_model.predict(X_test)))
print("Random Forest 15 estimators\n",classification_report(y_test, y_pred =rf_model.predict(X_test)))

#### 5-2. Reports: Original, imbalanced test data
---

In [ ]:
X_original = X.copy()
y_original = y.copy()
X_tr, X_te, y_tr, y_te = train_test_split(X_original, y_original, test_size = 0.25, random_state = 23)

In [ ]:
sns.countplot(y_te)

In [ ]:
print("Logistic Regression\n",classification_report(y_te, y_pred =LogisticReg.predict(X = X_te)))
print("Decision Tree\n",classification_report(y_te, y_pred =dt_model.predict(X_te)))
print("Random Forest 15 estimators\n",classification_report(y_te, y_pred =rf_model.predict(X_te)))

In [ ]:
print("=====Score of each model=========")
print("Logistic Regression: ",LogisticReg.score(X = X_te, y = y_te))
print("Decision Tree: ",dt_model.score(X = X_te, y = y_te))
print("Random Forest:",rf_model.score(X = X_te, y = y_te))

#### 6. Changing thresholds
---

In [ ]:
from sklearn.metrics import accuracy_score
thresholds = np.arange(0, 1, 0.1)
accuracies_dt = []
y_pred_prob = dt_model.predict_proba(X = X_te)
for i in thresholds:
    y_pred = y_pred_prob[:,1]>i
    accuracies_dt.append(accuracy_score(y_pred = y_pred , y_true = y_te))
accuracies_dt

In [ ]:
plt.figure() 
plt.scatter(thresholds, accuracies_dt )
plt.xlabel("Thresholds", fontsize = 13, color = 'black')
plt.ylabel("Accuracy score", fontsize = 13, color = 'black')
plt.ylim([0,1])
plt.suptitle("Thresholds change, Decision Tree", fontsize = 20, color = "black")


In [ ]:
thresholds = np.arange(0, 1, 0.1)
accuracies_rf = []
y_pred_prob = rf_model.predict_proba(X = X_te)
for i in thresholds:
    y_pred = y_pred_prob[:,1]>i
    accuracies_rf.append(accuracy_score(y_pred = y_pred , y_true = y_te))
accuracies_rf

In [ ]:
plt.figure() 
plt.scatter(thresholds, accuracies_rf )
plt.xlabel("Thresholds", fontsize = 13, color = 'black')
plt.ylabel("Accuracy score", fontsize = 13, color = 'black')
plt.ylim([0,1])
plt.suptitle("Thresholds change, Random Forest", fontsize = 20, color = "black")

#### 7. Conclusion
---

### Hm.... came out unexpected high scores, did I miss something...???